# DSE 220 Homework 4 

### Load Libraries 

In [2]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.corpus import brown
from nltk.corpus import stopwords
from collections import Counter
from math import log

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score


### Q1
Load data.

In [3]:
# Brown_words. 
text = brown.words()

# Convert to lowercase.
lower_words = [word.lower() for word in text]

### Q2
Data Preparation.

In [3]:
# Remove stop words and punctuations. 
stop_words = stopwords.words('english')
punctuations = list(string.punctuation)
useless_words = stop_words + punctuations
# words = [word for word in lower_words if word not in useless_words]
words = [word for word in lower_words if word not in useless_words and word.isalnum()]

In [4]:
# Number of words after removing stopwords and punctuations.
print('Number of words after cleaning:',len(words))

Number of words after cleaning: 515882


In [5]:
# Counting frequencies of words.
word_count = Counter(words)

# 5000 of most commonly-occuring words.
V_counts_dict = dict(word_count.most_common()[:5000])
V = list(V_counts_dict.keys())

# Context words: 1000 of most commonly-occuring words.
C_counts_dict = dict(word_count.most_common()[:1000])
C = list(C_counts_dict.keys())


### Q3

Overall distribution of context words (Distributions over C):

$Pr(c)=\frac{count(c_{j})}{count(c)}$

In [6]:
def Pr_c(C_counts_dict):
    """
    Overall distribution of context words.
    """
    total_C_counts = sum(C_counts_dict.values())
    pr_c_dict = dict([(k, v/total_C_counts) for k,v in C_counts_dict.items()])
    
    return pr_c_dict

In [7]:
# Pr(c)
pr_c_dict = Pr_c(C_counts_dict)

Probability of context words around $w$: 

$Pr(c|w)=\frac{count(c,w)}{count(w)}$

In [8]:
def WINDOW(index):
    """
    Get surrounding window of four words indices.
    """
    return [index-2,index-1,index+1,index+2]      

In [9]:

def COUNT(words,V,C):
    """
    Count the number of times c occurs in a window around word w.
    """
    CW_counts_dict={}
    
    for w in V:
        CW_counts_dict[w]={}

        for c in C:
            CW_counts_dict[w][c]=0
                    
    for index in range(len(words)):
        if words[index] in V:
            w = words[index]            
            window_index = WINDOW(index)
            
            for item in window_index:
                if item in range(len(words)):
                    if words[item] in C:
                        c = words[item]
                        CW_counts_dict[w][c]+=1
    
    return CW_counts_dict


In [10]:
def Pr_cw(V_counts_dict, CW_counts_dict):
    """
    Probability of context words c around w. 
    """
    # Probability of 
    pr_cw_dict={}
    for w in V:
        pr_cw_dict[w]={}
        for c in C:
            pr_cw_dict[w][c]=CW_counts_dict[w][c]/V_counts_dict[w]   
    
    return pr_cw_dict


In [11]:
# Pr(c|w)
n_wc_dict = COUNT(words,V,C)
pr_cw_dict = Pr_cw(V_counts_dict, n_wc_dict)

In [12]:
# n_wc_df = pd.DataFrame(n_wc_dict).T
# n_wc_df.head(5)

### Q4

Represent each vocabulary item w by a |C|-dimensional vector $\phi(w)$:

$\phi(w)=max(0, log\frac{Pr(c|w)}{Pr(c)})$

In [13]:
def Phi(pr_cw,pr_c):
    phi={}
    for w in V:
        phi[w]={}
        for c in C:
            if pr_cw[w][c]/pr_c[c]>1:
                phi[w][c]=log(pr_cw[w][c]/pr_c[c])
            else:
                phi[w][c]=0            
    return phi

In [14]:
# Dictionary of phi_w.
phi=Phi(pr_cw_dict,pr_c_dict)

In [15]:
# Convert phi_w dictionary to dataframe(5000*1000). 
phi_df = pd.DataFrame(phi).T

### Q5:  
100-dimentional representation.

In [16]:
pca = PCA(n_components=100)
phi_transformed = pca.fit_transform(phi_df)
phi_transformed_df = pd.DataFrame(phi_transformed,index=V)

In [17]:
phi_transformed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
one,9.943482,4.068119,-0.399103,1.394123,-2.121217,-2.034808,0.099559,0.613726,-0.223209,1.133620,...,0.093434,-0.122450,-0.569844,0.267297,0.221468,-0.219818,-0.049514,0.356502,0.241374,-0.495892
would,10.541195,3.996600,-3.114096,-0.164777,-3.244383,4.727616,0.218133,-1.578871,-0.295157,1.738129,...,0.270259,-0.963589,0.384445,0.459622,-1.036262,1.433826,-1.132137,-0.187011,-0.121513,-0.308771
said,7.509495,10.703473,-2.160421,-2.972781,-3.415102,3.501724,1.485335,1.165958,-1.091091,-0.531963,...,0.290679,0.016524,-0.337564,0.292447,-0.362312,-0.346557,0.202809,-0.958901,0.025326,-0.284423
new,9.155203,-3.794313,2.931107,-3.927246,2.010811,-2.223109,-1.343689,1.194033,0.875259,1.564110,...,-1.379819,0.018407,0.850753,-0.559948,-1.451546,0.819477,1.305234,-0.721986,-0.732835,0.390011
could,9.741721,8.684384,-3.133943,0.570771,-1.532481,2.746958,1.175994,-0.080773,-1.344955,1.297109,...,-0.067938,-1.308629,-0.586168,-0.121710,-0.940357,0.573011,-0.379509,-0.232038,0.441013,0.174857


### Q6
### 1. Nearest Neighbor

In [18]:
def NearestNeighbor(TestWord, WordsVectorDf, distance):
    
    Distance={}
    
    for i in WordsVectorDf.index:
        if i!=TestWord:
            
            X = np.array(WordsVectorDf.loc[TestWord,:]).reshape(1,-1)
            Y = np.array(WordsVectorDf.loc[i,:]).reshape(1,-1)
            Distance[i] = pairwise_distances(X,Y,metric=distance)
            
    min_distance = min(Distance, key=Distance.get)
    
    return min_distance
            

#### NN - cosine distance

In [22]:
# List of 25 test words.
test_words_list = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind', 'africa', 'chicago', 
                   'revolution', 'september', 'chemical', 'detergent', 'dictionary', 'storm', 'worship',
                   'school', 'weather', 'tax', 'art','data', 'america', 'earth','happy','female','color','flight']

In [23]:
# Nearest Neighbors of test words using cosine distance.
for TestWord in test_words_list:
    NN = NearestNeighbor(TestWord, phi_transformed_df, 'cosine')
    print('Nearest neighbor: {} -- {}'.format(TestWord, NN))

Nearest neighbor: communism -- utopian
Nearest neighbor: autumn -- slide
Nearest neighbor: cigarette -- smelled
Nearest neighbor: pulmonary -- artery
Nearest neighbor: mankind -- civil
Nearest neighbor: africa -- asia
Nearest neighbor: chicago -- portland
Nearest neighbor: revolution -- exercise
Nearest neighbor: september -- december
Nearest neighbor: chemical -- milligrams
Nearest neighbor: detergent -- cleaning
Nearest neighbor: dictionary -- text
Nearest neighbor: storm -- autumn
Nearest neighbor: worship -- religion
Nearest neighbor: school -- college
Nearest neighbor: weather -- hot
Nearest neighbor: tax -- income
Nearest neighbor: art -- literature
Nearest neighbor: data -- results
Nearest neighbor: america -- states
Nearest neighbor: earth -- dust
Nearest neighbor: happy -- know
Nearest neighbor: female -- feature
Nearest neighbor: color -- dress
Nearest neighbor: flight -- landing


#### NN - euclidean distance

In [24]:
# Nearest Neighbors of test words using cosine distance.
for TestWord in test_words_list:
    NN = NearestNeighbor(TestWord, phi_transformed_df, 'euclidean')
    print('Nearest neighbor: {} -- {}'.format(TestWord, NN))

Nearest neighbor: communism -- utopian
Nearest neighbor: autumn -- slide
Nearest neighbor: cigarette -- smelled
Nearest neighbor: pulmonary -- artery
Nearest neighbor: mankind -- fatal
Nearest neighbor: africa -- asia
Nearest neighbor: chicago -- portland
Nearest neighbor: revolution -- twentieth
Nearest neighbor: september -- december
Nearest neighbor: chemical -- milligrams
Nearest neighbor: detergent -- fabrics
Nearest neighbor: dictionary -- text
Nearest neighbor: storm -- autumn
Nearest neighbor: worship -- voluntary
Nearest neighbor: school -- college
Nearest neighbor: weather -- cheap
Nearest neighbor: tax -- income
Nearest neighbor: art -- english
Nearest neighbor: data -- include
Nearest neighbor: america -- peoples
Nearest neighbor: earth -- dust
Nearest neighbor: happy -- wondered
Nearest neighbor: female -- nude
Nearest neighbor: color -- pure
Nearest neighbor: flight -- landing


### 2. Clustering

### KMeans clustering

In [25]:
X = phi_transformed_df

In [32]:
# Initialize k-means
k_means = KMeans(algorithm='full', n_clusters=100,init='k-means++',random_state=20)
# Fit the k-means by passing it the input
k_means.fit(X)

KMeans(algorithm='full', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=20, tol=0.0001, verbose=0)

In [34]:
# Get the cluster labels.
kmeans_labels = k_means.labels_
# Calculate the silhouette score and davies bouldin score.
print('Kmeans:\n')
print('silhouette_score=',silhouette_score(X,kmeans_labels, metric='euclidean'))

# Convert word and cluster to a dataframe.
data = {'word': V,'cluster': kmeans_labels}
kmeans_word_cluster_df = pd.DataFrame(data)
# Print all words in each cluster. 
print('\nNumber of words in each cluster:\n')
print(np.array(kmeans_word_cluster_df.groupby('cluster')['word'].count()),'\n')

Kmeans:

silhouette_score= -0.01705429050979716

Number of words in each cluster:

[ 53   5  92  93  31  17  54  32  19  91  21  72  18 125  52  77  30  46
  37  45  75  23  54  14  35  48  99  33  30  86 111  63  95  18  42  77
  22  29  53  34  45 105  52  40  14  44   9  14  50 116  51  37  88  51
  62  24   6  93  37  27  65  80  19  50  40  11  24  12 112  51  82  57
  57  38  97 191  68   1  14  15  19  18  68  17  83 197  41  16  83  65
  19  40  16  19  29  32  68  26  23  21] 



### Agglomerative clustering

In [35]:
linkage_list = ['ward','average','complete','single']
labels={}
print('cosine distance:\n')
for linkage in linkage_list:
    if linkage != 'ward':
        model = AgglomerativeClustering(n_clusters=100,affinity='cosine',linkage=linkage)
        # Get the cluster labels.
        labels[('cosine',linkage)] = model.fit_predict(X)
        # Calculate the silhouette score and davies bouldin score.
        print(linkage,'silhouette_score=',silhouette_score(X,labels[('cosine',linkage)], metric='euclidean'))

print('\n euclidean distance:\n')
for linkage in linkage_list:
    model = AgglomerativeClustering(n_clusters=100,affinity='euclidean',linkage=linkage)
    # Get the cluster labels.
    labels[('euclidean', linkage)] = model.fit_predict(X)
    # Calculate the silhouette score and davies bouldin score.
    print(linkage,'silhouette_score=',silhouette_score(X,labels[('euclidean', linkage)], metric='euclidean'))

   

cosine distance:

average silhouette_score= -0.029540530632476737
complete silhouette_score= -0.04568668896804509
single silhouette_score= -0.1803078951213885

 euclidean distance:

ward silhouette_score= -0.027399593772126526
average silhouette_score= 0.0716034046173084
complete silhouette_score= 0.0005346767937223774
single silhouette_score= 0.023402049419600342


**Paremeters: n_clusters=100,  affinity='euclidean',  linkage='complete'**

In [42]:
# Convert word and cluster to a dataframe.
data = {'word': V,'cluster': labels['euclidean','complete']}
word_cluster_df = pd.DataFrame(data)
# Print all words in each cluster. 
print('Number of words in each cluster:\n')
print(np.array(word_cluster_df.groupby('cluster')['word'].count()),'\n')
# print(word_cluster_df.groupby('cluster')['word'].apply(list))

Number of words in each cluster:

[  14   24   68   26   18   25   78   28  230   19   18   22  147 1855
   28   17   14   63   48   36   20    9   10    8   33   20   28    9
    6    7   32  291   74   19   25   12   27   27   54   11   12   46
   11   96   10    4   13    3   44   21   17   95   64   70   16    9
    5    2   11   11   13    9   10   13   24   15    3   12   16   19
   33    1    4    3   36    8  528    5   14   33    3   18   10   14
   10    2    4    2    7    5    8    8    2    8    3    4   10   12
    6    5] 



Analysis: Overfitting

### Selected KMeans algorithm which using euclidean distance for the clustering. 

In [43]:
# print out every words in each cluster.
cluster_dict = kmeans_word_cluster_df.groupby('cluster')['word'].apply(list).to_dict()
print('KMeans:\n')
for cluster in range(100):
    print('Label: {}: {}\n'.format(cluster, cluster_dict[cluster]))

KMeans:

Label: 0: ['places', 'names', 'dog', 'boat', 'forth', 'signs', 'families', 'pictures', 'ends', 'score', 'passing', 'bodies', 'dream', 'unusual', 'guests', 'songs', 'negroes', 'orders', 'birds', 'appearance', 'notes', 'double', 'divided', 'games', 'advice', 'colors', 'brilliant', 'brothers', 'neighbors', 'furniture', 'license', 'bomb', 'cards', 'crew', 'visitors', 'columns', 'eggs', 'roll', 'seldom', 'corn', 'kids', 'volunteers', 'shots', 'suits', 'rifles', 'grades', 'carries', 'hits', 'pitch', 'joke', 'passengers', 'clock', 'salesmen']

Label: 1: ['point', 'clear', 'simple', 'decided', 'statement']

Label: 2: ['associated', 'chain', 'resistance', 'fears', 'bond', 'rapid', 'muscle', 'reactions', 'distinct', 'limits', 'exception', 'proof', 'studying', 'intense', 'controlled', 'maturity', 'experimental', 'represent', 'tremendous', 'outstanding', 'magic', 'devices', 'intention', 'phenomenon', 'stems', 'precise', 'awareness', 'fallout', 'juniors', 'worker', 'cuts', 'stable', 'signa